In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2

import os
from datetime import datetime
import sys
import time

sys.path.insert(0, "../scripts/")

from utils import train_test_split, image_batch_generator, get_train_augmentation, random_batch_generator, get_table_augmentation
from utils import DATASET_PATH, DS_IMAGES, DS_MASKS, SaveValidSamplesCallback
import utils
from metrics import iou, f1_score, jaccard_distance
import metrics
from vis import anshow, imshow
import vis
from models import TableNet, load_unet_model

IMAGE_NAMES = os.listdir(DS_IMAGES)

import importlib
importlib.reload(metrics)
importlib.reload(vis)
importlib.reload(utils)

<module 'utils' from '/home/tigran-analysed/Desktop/analysed.ai/table_extractor/notebooks/../scripts/utils.py'>

In [2]:
model = load_unet_model((512, 512), 2, weight_decay=0.1)
optim = tf.keras.optimizers.Adam()

2022-08-15 15:07:19.522588: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/tigran-analysed/Desktop/analysed.ai/table_extractor/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-08-15 15:07:19.522664: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-15 15:07:19.522715: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tigrananalysed-System-Product-Name): /proc/driver/nvidia/version does not exist
2022-08-15 15:07:19.523228: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlo

In [3]:
checkpoint = tf.train.Checkpoint(step=tf.Variable(1), optimizer=tf.keras.optimizers.Adam(), net=model)
# print(f"loading checkpoint {'training_checkpoints/' + '2022.07.30-22/' + 'ckpt-238'}")
status = checkpoint.restore("../checkpoints/ckpt-624")
status

In [5]:
loss_fn = jaccard_distance

train_names, valid_names = train_test_split(IMAGE_NAMES, shuffle=True, random_state=2022, test_size=0.2)

In [6]:
len(valid_names)

805

In [7]:
valid_batch_generator = image_batch_generator(
                            valid_names, 
                            batch_size=1, 
                            resize_shape=(512, 512),
                            aug_transform=None,
                            normalize=True, include_edges_as_band=True
                        )

In [8]:
def print_progress(name, metrics, step, all_steps):
    str_prog = f"{all_steps}/{step}: "
    str_prog += "{} loss {:.4f}, tf_iou {:.4f}, iou {:.4f}, f1 {:.4f}, prec {:.4f}, rec {:.4f}".format(
        name,
        np.mean(metrics["loss"]),
        np.mean(metrics["tf_iou"]),
        np.mean(metrics["iou"]), 
        np.mean(metrics["f1"]),
        np.mean(metrics["precision"]),
        np.mean(metrics["recall"])
    )

    print(str_prog, end='\r')

In [9]:
batch_X, batch_y = utils.read_inf_sample(valid_names[:50], (512, 512))

In [19]:
val_metrics = {n:[] for n in ("loss", "iou", "tf_iou", "f1", "precision", "recall")}
mean_time = []

for i, (X, y, name) in enumerate(zip(batch_X, batch_y, valid_names)):

    start = time.time()

    raw = model(tf.expand_dims(X, 0), training=False)
    raw = tf.squeeze(raw)

    pred1 = utils.preprocess_raw_output(raw, 2, 1000)
    pred2 = utils.preprocess_raw_output(pred1, 2, 0, max_seg_dist=50)
    # pred2 = utils.preprocess_raw_output(pred2, 2, 0, max_seg_dist=20)
    # pred2 = utils.preprocess_raw_output(pred2, 2, 0)

    mean_time.append(time.time() - start)
    # print(y.shape, pred.shape)

    loss_value = loss_fn(tf.expand_dims(y, 0), tf.expand_dims(pred2, 0))
    (
        iou_value, tf_iou_value,
        f1_score_value,
        presicion_value, 
        recall_value
    ) = metrics.calculate_metrics([y], [pred2])

    val_metrics["loss"].append(np.mean(loss_value))
    val_metrics["iou"].append(iou_value)
    val_metrics["tf_iou"].append(tf_iou_value)
    val_metrics["f1"].append(f1_score_value)
    val_metrics["precision"].append(presicion_value)
    val_metrics["recall"].append(recall_value)

    # print(X[:, :, 0].shape, raw.shape, y.shape, pred.shape)
    final_img = cv2.hconcat([
        np.array(X[:, :, 0] * 255, dtype=np.uint8),
        np.array(raw * 255, dtype=np.uint8),
        np.array(y * 255, dtype=np.uint8),
        np.array(pred1 * 255, dtype=np.uint8),
        np.array(pred2 * 255, dtype=np.uint8)
        # np.array(thresh)
    ])

    b_n, ext = os.path.splitext(name)
    cv2.imwrite("pred_samples/{:.4f}_{}.png".format(iou_value, name), final_img)
    # break